In [4]:
import pandas as pd
data = pd.read_csv('generated_reviews.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3542 entries, 0 to 3541
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   gen_text  3542 non-null   object
 1   category  3542 non-null   object
 2   label     3542 non-null   object
dtypes: object(3)
memory usage: 83.1+ KB


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# 데이터 불러오기
data = pd.read_csv('korean_review_final.csv')  # 데이터 파일 경로 수정

# 레이블 인코딩
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label'])

# 텍스트와 레이블 분리
texts = data['text']
labels = data['label_encoded']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# 텍스트 토크나이징 및 패딩
max_words = 10000  # 사용할 최대 단어 수
max_len = 100  # 시퀀스의 최대 길이

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# CNN 모델 구축
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))  # 임베딩 레이어
model.add(Conv1D(128, 3, activation='relu'))  # CNN 레이어
model.add(GlobalMaxPooling1D())  # Global Max Pooling
model.add(Dropout(0.5))  # Dropout
model.add(Dense(16, activation='relu'))  # 완전 연결층
model.add(Dense(1, activation='sigmoid'))  # 출력층 (이진 분류)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    X_train_pad, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test_pad, y_test)
)

# 모델 평가
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/5


C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


342/342 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9431 - loss: 0.1832 - val_accuracy: 0.9938 - val_loss: 0.0175
Epoch 2/5
342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9973 - loss: 0.0093 - val_accuracy: 0.9952 - val_loss: 0.0128
Epoch 3/5
342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9993 - loss: 0.0036 - val_accuracy: 0.9963 - val_loss: 0.0151
Epoch 4/5
342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9996 - loss: 0.0021 - val_accuracy: 0.9963 - val_loss: 0.0147
Epoch 5/5
342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9997 - loss: 0.0016 - val_accuracy: 0.9965 - val_loss: 0.0161
342/342 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9945 - loss: 0.0242 
Test Accuracy: 99.65%


In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

# CNN 모델 학습 함수
def train_cnn(filters, kernel_size, dropout_rate, dense_units, batch_size, epochs):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))  # 임베딩 레이어
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))  # CNN 레이어
    model.add(GlobalMaxPooling1D())  # Global Max Pooling
    model.add(Dropout(dropout_rate))  # Dropout
    model.add(Dense(dense_units, activation='relu'))  # 완전 연결층
    model.add(Dense(1, activation='sigmoid'))  # 출력층
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # EarlyStopping으로 과적합 방지
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    # 모델 학습
    model.fit(
        X_train_pad, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test_pad, y_test),
        callbacks=[early_stopping],
        verbose=0
    )
    
    # 검증 데이터 정확도 계산
    y_pred = (model.predict(X_test_pad) > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# 하이퍼파라미터 값 설정
filters_list = [64, 128]
kernel_size_list = [3, 5]
dropout_rate_list = [0.3, 0.5]
dense_units_list = [32, 64]
batch_size_list = [32, 64]
epochs_list = [3, 5]

# 최적 하이퍼파라미터 탐색
best_accuracy = 0
best_params = {}

for filters in filters_list:
    for kernel_size in kernel_size_list:
        for dropout_rate in dropout_rate_list:
            for dense_units in dense_units_list:
                for batch_size in batch_size_list:
                    for epochs in epochs_list:
                        print(f"Training with filters={filters}, kernel_size={kernel_size}, dropout_rate={dropout_rate}, "
                              f"dense_units={dense_units}, batch_size={batch_size}, epochs={epochs}")
                        
                        accuracy = train_cnn(filters, kernel_size, dropout_rate, dense_units, batch_size, epochs)
                        print(f"Accuracy: {accuracy:.4f}")
                        
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params = {
                                'filters': filters,
                                'kernel_size': kernel_size,
                                'dropout_rate': dropout_rate,
                                'dense_units': dense_units,
                                'batch_size': batch_size,
                                'epochs': epochs
                            }

# 최적 하이퍼파라미터 출력
print("Best Accuracy:", best_accuracy)
print("Best Hyperparameters:", best_params)


Training with filters=64, kernel_size=3, dropout_rate=0.3, dense_units=32, batch_size=32, epochs=3
342/342 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step
Accuracy: 0.9959
Training with filters=64, kernel_size=3, dropout_rate=0.3, dense_units=32, batch_size=32, epochs=5


C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


342/342 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Accuracy: 0.9969
Training with filters=64, kernel_size=3, dropout_rate=0.3, dense_units=32, batch_size=64, epochs=3


C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


342/342 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step
Accuracy: 0.9964
Training with filters=64, kernel_size=3, dropout_rate=0.3, dense_units=32, batch_size=64, epochs=5


...

342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step  
Accuracy: 0.9961
Training with filters=128, kernel_size=5, dropout_rate=0.5, dense_units=64, batch_size=64, epochs=3


C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step  
Accuracy: 0.9968
Training with filters=128, kernel_size=5, dropout_rate=0.5, dense_units=64, batch_size=64, epochs=5


C:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step  
Accuracy: 0.9963
Best Accuracy: 0.997165584712444
Best Hyperparameters: {'filters': 128, 'kernel_size': 3, 'dropout_rate': 0.5, 'dense_units': 32, 'batch_size': 32, 'epochs': 3}
